In [14]:
import pandas as pd

In [15]:
bat_df = pd.read_csv('../Datasets/batter_stats.csv')
pitch_df = pd.read_csv('../Datasets/pitcher_stats.csv')
lineups = pd.read_csv('../Datasets/Lineups_2021_06_09.csv')

In [16]:
# The dots in names like J.T. were inconsistent across datasets
bat_df.loc[:,'Name'] = bat_df['Name'].str.replace('.', '')
pitch_df.loc[:,'Name'] = pitch_df['Name'].str.replace('.', '')
lineups.loc[:,' player name'] = lineups[' player name'].str.replace('.', '')

In [17]:
bats_r = bat_df[bat_df['bats'] == 'R']
bats_l = bat_df[bat_df['bats'] == 'L']

In [18]:
# Making name column consistent
lineups.columns = lineups.columns.str.replace(' player name', 'Name')

In [19]:
# Individual name that was giving trouble
lineups.loc[lineups['Name'] == 'Jonathan Gray', 'Name'] = 'Jon Gray'

In [20]:
# Creates a list of teams with no repeats
# The order needs to be kept from the dataframe because 
# opposing teams are paired
team_list = []
for team in lineups['team code']:
    if team not in team_list:
        team_list.append(team)
# Creating a dictionary of every team, and the team they face
matchups = {}
for i in range(1, len(team_list), 2):
    matchups[team_list[i]] = team_list[i-1]
    matchups[team_list[i-1]] = team_list[i]

In [21]:
lineups

,team code,game_date,game_number,mlb id,Name,batting order,confirmed,position
0,SF,6/9/2021,1,664774,LaMonte Wade,1,N,NaN
1,SF,6/9/2021,1,457763,Buster Posey,2,N,NaN
2,SF,6/9/2021,1,543105,Alex Dickerson,3,N,NaN
3,SF,6/9/2021,1,527038,Wilmer Flores,4,N,NaN
4,SF,6/9/2021,1,543063,Brandon Crawford,5,N,NaN
...,...,...,...,...,...,...,...,...
281,LAA,6/9/2021,1,670351,Jose Rojas,6,N,NaN
282,LAA,6/9/2021,1,642221,Kean Wong,7,N,NaN
283,LAA,6/9/2021,1,621532,Anthony Bemboom,8,N,NaN
284,LAA,6/9/2021,1,664058,David Fletcher,9,N,NaN


In [22]:
batters = lineups.loc[(lineups[' batting order'] != 'SP') & (lineups[' game_number'] == 1)]

In [23]:
pitch_df.drop(456, inplace = True)

In [24]:
matchups

{'TEX': 'SF',
 'SF': 'TEX',
 'OAK': 'ARI',
 'ARI': 'OAK',
 'SD': 'CHC',
 'CHC': 'SD',
 'PIT': 'LAD',
 'LAD': 'PIT',
 'PHI': 'ATL',
 'ATL': 'PHI',
 'BAL': 'NYM',
 'NYM': 'BAL',
 'CIN': 'MIL',
 'MIL': 'CIN',
 'BOS': 'HOU',
 'HOU': 'BOS',
 'TB': 'WAS',
 'WAS': 'TB',
 'MIA': 'COL',
 'COL': 'MIA',
 'DET': 'SEA',
 'SEA': 'DET',
 'CHW': 'TOR',
 'TOR': 'CHW',
 'MIN': 'NYY',
 'NYY': 'MIN',
 'STL': 'CLE',
 'CLE': 'STL',
 'LAA': 'KC',
 'KC': 'LAA'}

In [25]:
pitch_df.loc[pitch_df['Name']== 'Chase DeJong', 'throws']

Series([], Name: throws, dtype: object)

In [26]:
opp_pitch_hand = []
opp_pitcher = []
del_indexes = []
keep_pitch = []
for i, row in batters.iterrows():
    if len(lineups.loc[(lineups['team code'] == matchups[row['team code']])\
                            & (lineups[' batting order'] == 'SP') & (lineups[' game_number'] == 1), 'Name'].values) > 0: 
        opp_pitch = lineups.loc[(lineups['team code'] == matchups[row['team code']])\
                                & (lineups[' batting order'] == 'SP') & (lineups[' game_number'] == 1), 'Name'].values[0]
        if len(pitch_df.sort_values(by = 'IP', ascending = False).loc[pitch_df['Name'] == opp_pitch, 'throws'].values) > 0:
                opp_pitch_hand.append(pitch_df.sort_values(by = 'IP', ascending = False).loc[pitch_df['Name'] == opp_pitch, 'throws'].values[0])
                opp_pitcher.append(opp_pitch)
                keep_pitch.append(opp_pitch)
        else:
            del_indexes.append(i)
    else:
        del_indexes.append(i)
batters.drop(index = del_indexes, inplace = True)
batters['opp_pitch_hand'] = opp_pitch_hand
batters['opp_pitch'] = opp_pitcher


/Users/nperry725/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
<ipython-input-26-a4b56b56add5>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batters['opp_pitch_hand'] = opp_pitch_hand
<ipython-input-26-a4b56b56add5>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.htm

In [27]:
del_indexes

[29, 30, 31, 32, 33, 34, 35, 36, 37, 257, 258, 259, 260, 261, 262, 263, 264]

In [28]:
bats = []
k_pct = []
bb_pct = []
bb_k = []
bat_avg = []
obp = []
slg = []
ops = []
iso = []
babip = []
wrc = []
wraa = []
woba = []
wrc_plus = []
for i, row in batters.iterrows():
    # Pulling stats based on handedness of pitcher
    if row['opp_pitch_hand'] == 'R':
        batter_df = bat_df.loc[(bat_df['pitch']== 'R')&(bat_df['Name']==row['Name'])]
        # Checking for switch hitters
        if len(batter_df) == 2:
            # A switch hitter will hit using the opposite hand of the pitcher
            batter_df = batter_df.loc[batter_df['bats']== 'L']
    if row['opp_pitch_hand'] == 'L':
        batter_df = bat_df.loc[(bat_df['pitch']== 'L')&(bat_df['Name']==row['Name'])]
        # Checking for switch hitters
        if len(batter_df) == 2:
            # A switch hitter will hit using the opposite hand of the pitcher
            batter_df = batter_df.loc[batter_df['bats']== 'R']
    if len(batter_df) == 0:
        for col in batter_df.columns:
            batter_df[col] = [float('NaN')]
    bats.append(batter_df['bats'].values[0])
    k_pct.append(batter_df['K%'].values[0])
    bb_pct.append(batter_df['BB%'].values[0])
    bb_k.append(batter_df['BB/K'].values[0])
    bat_avg.append(batter_df['AVG'].values[0])
    obp.append(batter_df['OBP'].values[0])
    slg.append(batter_df['SLG'].values[0])
    ops.append(batter_df['OPS'].values[0])
    iso.append(batter_df['ISO'].values[0])
    babip.append(batter_df['BABIP'].values[0])
    wrc.append(batter_df['wRC'].values[0])
    wraa.append(batter_df['wRAA'].values[0])
    woba.append(batter_df['wOBA'].values[0])
    wrc_plus.append(batter_df['wRC+'].values[0])
batters.loc[:,'bats'] = bats
batters.loc[:,'k_pct'] = k_pct
batters.loc[:,'bb_pct'] = bb_pct
batters.loc[:,'bb_k'] = bb_k
batters.loc[:,'bat_avg'] = bat_avg
batters.loc[:,'obp'] = obp
batters.loc[:,'slg'] = slg
batters.loc[:,'ops'] = ops
batters.loc[:,'iso'] = iso
batters.loc[:,'babip'] = babip
batters.loc[:,'wrc'] = wrc
batters.loc[:,'wraa'] = wraa
batters.loc[:,'woba'] = woba
batters.loc[:,'wrc_plus'] = wrc_plus

<ipython-input-28-0499cefeeb18>:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batter_df[col] = [float('NaN')]
/Users/nperry725/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/Users/nperry725/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1745: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] =

In [29]:
batters

,team code,game_date,game_number,mlb id,Name,batting order,confirmed,position,opp_pitch_hand,opp_pitch,...,bat_avg,obp,slg,ops,iso,babip,wrc,wraa,woba,wrc_plus
0,SF,6/9/2021,1,664774,LaMonte Wade,1,N,NaN,R,Kyle Gibson,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,SF,6/9/2021,1,457763,Buster Posey,2,N,NaN,R,Kyle Gibson,...,0.287356,0.380000,0.528736,0.908736,0.241379,0.296875,18.355418,6.696081,0.393171,153.781748
2,SF,6/9/2021,1,543105,Alex Dickerson,3,N,NaN,R,Kyle Gibson,...,0.260163,0.324723,0.487805,0.812528,0.227642,0.278689,39.869358,6.869752,0.344193,119.683917
3,SF,6/9/2021,1,527038,Wilmer Flores,4,N,NaN,R,Kyle Gibson,...,0.252475,0.324444,0.435644,0.760088,0.183168,0.259036,29.916481,2.378455,0.329070,109.037629
4,SF,6/9/2021,1,543063,Brandon Crawford,5,N,NaN,R,Kyle Gibson,...,0.275304,0.348921,0.534413,0.883334,0.259109,0.313253,46.317552,12.510727,0.370748,136.804643
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
280,LAA,6/9/2021,1,501571,Juan Lagares,5,N,NaN,R,Brad Keller,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
281,LAA,6/9/2021,1,670351,Jose Rojas,6,N,NaN,R,Brad Keller,...,0.180723,0.209302,0.325301,0.534604,0.144578,0.237288,4.667351,-5.359679,0.231390,46.415233
282,LAA,6/9/2021,1,642221,Kean Wong,7,N,NaN,R,Brad Keller,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
283,LAA,6/9/2021,1,621532,Anthony Bemboom,8,N,NaN,R,Brad Keller,...,0.244444,0.333333,0.400000,0.733333,0.155556,0.290323,6.817952,0.424500,0.323252,107.103389


In [30]:
k_9 = []
bb_9 = []
k_bb = []
hr_9 = []
k_pct = []
bb_pct = []
k_bb_pct = []
bat_avg = []
whip = []
babip = []
lob_pct = []
fip = []
xfip = []
for i, row in batters.iterrows():
    # Pulling stats based on handedness of pitcher
    pitcher_df = pitch_df.sort_values(by = 'IP', ascending = False).loc[(pitch_df['batter']== row['bats'])&(pitch_df['Name']==row['opp_pitch']),:]
    if len(pitcher_df) == 0:
        for col in pitcher_df.columns:
            pitcher_df[col] = [float('NaN')]
    k_9.append(pitcher_df['K/9'].values[0])
    bb_9.append(pitcher_df['BB/9'].values[0])
    k_bb.append(pitcher_df['K/BB'].values[0])
    hr_9.append(pitcher_df['HR/9'].values[0])
    k_pct.append(pitcher_df['K%'].values[0])
    bb_pct.append(pitcher_df['BB%'].values[0])
    k_bb_pct.append(pitcher_df['K-BB%'].values[0])
    bat_avg.append(pitcher_df['AVG'].values[0])
    whip.append(pitcher_df['WHIP'].values[0])
    babip.append(pitcher_df['BABIP'].values[0])
    lob_pct.append(pitcher_df['LOB%'].values[0])
    fip.append(pitcher_df['FIP'].values[0])
    xfip.append(pitcher_df['xFIP'].values[0])
batters.loc[:,'opp_k/9'] = k_9
batters.loc[:,'opp_bb/9'] = k_9
batters.loc[:,'opp_k/bb'] = k_9
batters.loc[:,'opp_hr/9'] = k_9
batters.loc[:,'opp_k_pct'] = k_9
batters.loc[:,'opp_bb_pct'] = k_9
batters.loc[:,'opp_k_bb_pct'] = k_9
batters.loc[:,'opp_avg'] = k_9
batters.loc[:,'opp_whip'] = k_9
batters.loc[:,'opp_babip'] = k_9
batters.loc[:,'opp_lob_pct'] = k_9
batters.loc[:,'opp_fip'] = k_9
batters.loc[:,'opp_xfip'] = k_9

/Users/nperry725/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/Users/nperry725/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1745: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


In [31]:
pitchers = lineups.loc[(lineups[' batting order'] == 'SP')]

In [32]:
pitchers = pitchers[pitchers['Name'].isin(keep_pitch)]

In [33]:
batters

,team code,game_date,game_number,mlb id,Name,batting order,confirmed,position,opp_pitch_hand,opp_pitch,...,opp_hr/9,opp_k_pct,opp_bb_pct,opp_k_bb_pct,opp_avg,opp_whip,opp_babip,opp_lob_pct,opp_fip,opp_xfip
0,SF,6/9/2021,1,664774,LaMonte Wade,1,N,NaN,R,Kyle Gibson,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,SF,6/9/2021,1,457763,Buster Posey,2,N,NaN,R,Kyle Gibson,...,9.121622,9.121622,9.121622,9.121622,9.121622,9.121622,9.121622,9.121622,9.121622,9.121622
2,SF,6/9/2021,1,543105,Alex Dickerson,3,N,NaN,R,Kyle Gibson,...,5.491525,5.491525,5.491525,5.491525,5.491525,5.491525,5.491525,5.491525,5.491525,5.491525
3,SF,6/9/2021,1,527038,Wilmer Flores,4,N,NaN,R,Kyle Gibson,...,9.121622,9.121622,9.121622,9.121622,9.121622,9.121622,9.121622,9.121622,9.121622,9.121622
4,SF,6/9/2021,1,543063,Brandon Crawford,5,N,NaN,R,Kyle Gibson,...,5.491525,5.491525,5.491525,5.491525,5.491525,5.491525,5.491525,5.491525,5.491525,5.491525
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
280,LAA,6/9/2021,1,501571,Juan Lagares,5,N,NaN,R,Brad Keller,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
281,LAA,6/9/2021,1,670351,Jose Rojas,6,N,NaN,R,Brad Keller,...,7.263959,7.263959,7.263959,7.263959,7.263959,7.263959,7.263959,7.263959,7.263959,7.263959
282,LAA,6/9/2021,1,642221,Kean Wong,7,N,NaN,R,Brad Keller,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
283,LAA,6/9/2021,1,621532,Anthony Bemboom,8,N,NaN,R,Brad Keller,...,7.263959,7.263959,7.263959,7.263959,7.263959,7.263959,7.263959,7.263959,7.263959,7.263959


In [34]:
num_L_bats = []
num_R_bats = []
for i, row in pitchers.iterrows():
    opp_lineup = batters.loc[(batters['team code'] == matchups[row['team code']]),:]
    find_batters = []
    # There are batters whos data was null due to lack of plate appearances but handedness is known
    for j, row2 in opp_lineup.iterrows():
        if row2['bats'] != 'R' and row2['bats'] != 'L':
            find_bats = bat_df.loc[bat_df['Name'] == row2['Name'],:]
            if len(find_bats) > 0:
                find_batters.append((j, find_bats['bats'].values[0]))
            # If batter handedness is unknown, right is more likely
            if len(find_bats) == 0:
                find_batters.append((j, 'R'))
    for handedness in find_batters:
        opp_lineup.loc[handedness[0], 'bats'] = handedness[1]
    try:
        num_L_bats.append(opp_lineup['bats'].value_counts()['L'])
    except:
        num_L_bats.append(0)
    try:
        num_R_bats.append(opp_lineup['bats'].value_counts()['R'])
    except:
        num_R_bats.append(0)
pitchers['num_L_bats'] = num_L_bats
pitchers['num_R_bats'] = num_R_bats
is_nl = []
for i, row in pitchers.iterrows():
    if row['num_L_bats'] + row['num_R_bats'] == 9:
        is_nl.append(0)
    else:
        is_nl.append(1)
pitchers['is_nl'] = is_nl

/Users/nperry725/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


In [35]:
k_9_l = []
bb_9_l = []
k_bb_l = []
hr_9_l = []
k_pct_l = []
bb_pct_l = []
k_bb_pct_l = []
avg_l = []
whip_l = []
babip_l = []
lob_pct_l = []
fip_l = []
xfip_l = []
k_9_r = []
bb_9_r = []
k_bb_r = []
hr_9_r = []
k_pct_r = []
bb_pct_r = []
k_bb_pct_r = []
avg_r = []
whip_r = []
babip_r = []
lob_pct_r = []
fip_r = []
xfip_r = []
# Engineering features
k_9_l_tot = []
bb_9_l_tot = []
k_bb_l_tot = []
hr_9_l_tot = []
k_pct_l_tot = []
bb_pct_l_tot = []
k_bb_pct_l_tot = []
avg_l_tot = []
whip_l_tot = []
babip_l_tot = []
lob_pct_l_tot = []
fip_l_tot = []
xfip_l_tot = []
k_9_r_tot = []
bb_9_r_tot = []
k_bb_r_tot = []
hr_9_r_tot = []
k_pct_r_tot = []
bb_pct_r_tot = []
k_bb_pct_r_tot = []
avg_r_tot = []
whip_r_tot = []
babip_r_tot = []
lob_pct_r_tot = []
fip_r_tot = []
xfip_r_tot = []
for i, row in pitchers.iterrows():
    # Pulling stats of pitcher against lefty hitters
    pitcher_df_l = pitch_df.sort_values(by = 'IP', ascending = False).loc[(pitch_df['batter']== 'L')&(pitch_df['Name']==row['Name']),:]
    if len(pitcher_df_l) == 0:
        for col in pitcher_df_l.columns:
            pitcher_df_l[col] = [float('NaN')]
    k_9_l.append(pitcher_df_l['K/9'].values[0])
    bb_9_l.append(pitcher_df_l['BB/9'].values[0])
    k_bb_l.append(pitcher_df_l['K/BB'].values[0])
    hr_9_l.append(pitcher_df_l['HR/9'].values[0])
    k_pct_l.append(pitcher_df_l['K%'].values[0])
    bb_pct_l.append(pitcher_df_l['BB%'].values[0])
    k_bb_pct_l.append(pitcher_df_l['K-BB%'].values[0])
    avg_l.append(pitcher_df_l['AVG'].values[0])
    whip_l.append(pitcher_df_l['WHIP'].values[0])
    babip_l.append(pitcher_df_l['BABIP'].values[0])
    lob_pct_l.append(pitcher_df_l['LOB%'].values[0])
    fip_l.append(pitcher_df_l['FIP'].values[0])
    xfip_l.append(pitcher_df_l['xFIP'].values[0])
    # Feature Engineering
    k_9_l_tot.append(pitcher_df_l['K/9'].values[0] * row['num_L_bats'])
    bb_9_l_tot.append(pitcher_df_l['BB/9'].values[0] * row['num_L_bats'])
    k_bb_l_tot.append(pitcher_df_l['K/BB'].values[0] * row['num_L_bats'])
    hr_9_l_tot.append(pitcher_df_l['HR/9'].values[0] * row['num_L_bats'])
    k_pct_l_tot.append(pitcher_df_l['K%'].values[0] * row['num_L_bats'])
    bb_pct_l_tot.append(pitcher_df_l['BB%'].values[0] * row['num_L_bats'])
    k_bb_pct_l_tot.append(pitcher_df_l['K-BB%'].values[0] * row['num_L_bats'])
    avg_l_tot.append(pitcher_df_l['AVG'].values[0] * row['num_L_bats'])
    whip_l_tot.append(pitcher_df_l['WHIP'].values[0] * row['num_L_bats'])
    babip_l_tot.append(pitcher_df_l['BABIP'].values[0] * row['num_L_bats'])
    lob_pct_l_tot.append(pitcher_df_l['LOB%'].values[0] * row['num_L_bats'])
    fip_l_tot.append(pitcher_df_l['FIP'].values[0] * row['num_L_bats'])
    xfip_l_tot.append(pitcher_df_l['xFIP'].values[0] * row['num_L_bats'])
    # Pulling stats against righty hitters
    pitcher_df_r = pitch_df.sort_values(by = 'IP', ascending = False).loc[(pitch_df['batter']== 'R')&(pitch_df['Name']== row['Name']),:]
    if len(pitcher_df_r) == 0:
        for col in pitcher_df_r.columns:
            pitcher_df_r[col] = [float('NaN')]
    k_9_r.append(pitcher_df_r['K/9'].values[0])
    bb_9_r.append(pitcher_df_r['BB/9'].values[0])
    k_bb_r.append(pitcher_df_r['K/BB'].values[0])
    hr_9_r.append(pitcher_df_r['HR/9'].values[0])
    k_pct_r.append(pitcher_df_r['K%'].values[0])
    bb_pct_r.append(pitcher_df_r['BB%'].values[0])
    k_bb_pct_r.append(pitcher_df_r['K-BB%'].values[0])
    avg_r.append(pitcher_df_r['AVG'].values[0])
    whip_r.append(pitcher_df_r['WHIP'].values[0])
    babip_r.append(pitcher_df_r['BABIP'].values[0])
    lob_pct_r.append(pitcher_df_r['LOB%'].values[0])
    fip_r.append(pitcher_df_r['FIP'].values[0])
    xfip_r.append(pitcher_df_r['xFIP'].values[0])
    # Feature Engineering
    k_9_r_tot.append(pitcher_df_r['K/9'].values[0] * row['num_R_bats'])
    bb_9_r_tot.append(pitcher_df_r['BB/9'].values[0] * row['num_R_bats'])
    k_bb_r_tot.append(pitcher_df_r['K/BB'].values[0] * row['num_R_bats'])
    hr_9_r_tot.append(pitcher_df_r['HR/9'].values[0] * row['num_R_bats'])
    k_pct_r_tot.append(pitcher_df_r['K%'].values[0] * row['num_R_bats'])
    bb_pct_r_tot.append(pitcher_df_r['BB%'].values[0] * row['num_R_bats'])
    k_bb_pct_r_tot.append(pitcher_df_r['K-BB%'].values[0] * row['num_R_bats'])
    avg_r_tot.append(pitcher_df_r['AVG'].values[0] * row['num_R_bats'])
    whip_r_tot.append(pitcher_df_r['WHIP'].values[0] * row['num_R_bats'])
    babip_r_tot.append(pitcher_df_r['BABIP'].values[0] * row['num_R_bats'])
    lob_pct_r_tot.append(pitcher_df_r['LOB%'].values[0] * row['num_R_bats'])
    fip_r_tot.append(pitcher_df_r['FIP'].values[0] * row['num_R_bats'])
    xfip_r_tot.append(pitcher_df_r['xFIP'].values[0] * row['num_R_bats'])
pitchers['k_9_l'] = k_9_l
pitchers['bb_9_l'] = bb_9_l
pitchers['k_bb_l'] = k_bb_l
pitchers['hr_9_l'] = hr_9_l
pitchers['k_pct_l'] = k_pct_l
pitchers['bb_pct_l'] = bb_pct_l
pitchers['k_bb_pct_l'] = k_bb_pct_l
pitchers['bat_avg_l'] = avg_l
pitchers['whip_l'] = whip_l
pitchers['babip_l'] = babip_l
pitchers['lob_pct_l'] = lob_pct_l
pitchers['fip_l'] = fip_l
pitchers['xfip_l'] = xfip_l
# Right handed batter stats
pitchers['k_9_r'] = k_9_r
pitchers['bb_9_r'] = bb_9_r
pitchers['k_bb_r'] = k_bb_r
pitchers['hr_9_r'] = hr_9_r
pitchers['k_pct_r'] = k_pct_r
pitchers['bb_pct_r'] = bb_pct_r
pitchers['k_bb_pct_r'] = k_bb_pct_r
pitchers['bat_avg_r'] = avg_r
pitchers['whip_r'] = whip_r
pitchers['babip_r'] = babip_r
pitchers['lob_pct_r'] = lob_pct_r
pitchers['fip_r'] = fip_r
pitchers['xfip_r'] = xfip_r
# feature engineered left handed batter stats
pitchers['k_9_l_tot'] = k_9_l_tot
pitchers['bb_9_l_tot'] = bb_9_l_tot
pitchers['k_bb_l_tot'] = k_bb_l_tot
pitchers['hr_9_l_tot'] = hr_9_l_tot
pitchers['k_pct_l_tot'] = k_pct_l_tot
pitchers['bb_pct_l_tot'] = bb_pct_l_tot
pitchers['k_bb_pct_l_tot'] = k_bb_pct_l_tot
pitchers['bat_avg_l_tot'] = avg_l_tot
pitchers['whip_l_tot'] = whip_l_tot
pitchers['babip_l_tot'] = babip_l_tot
pitchers['lob_pct_l_tot'] = lob_pct_l_tot
pitchers['fip_l_tot'] = fip_l_tot
pitchers['xfip_l_tot'] = xfip_l_tot
# feature engineered righ handed batter stats
pitchers['k_9_r_tot'] = k_9_r_tot
pitchers['bb_9_r_tot'] = bb_9_r_tot
pitchers['k_bb_r_tot'] = k_bb_r_tot
pitchers['hr_9_r_tot'] = hr_9_r_tot
pitchers['k_pct_r_tot'] = k_pct_r_tot
pitchers['bb_pct_r_tot'] = bb_pct_r_tot
pitchers['k_bb_pct_r_tot'] = k_bb_pct_r_tot
pitchers['bat_avg_r_tot'] = avg_r_tot
pitchers['whip_r_tot'] = whip_r_tot
pitchers['babip_r_tot'] = babip_r_tot
pitchers['lob_pct_r_tot'] = lob_pct_r_tot
pitchers['fip_r_tot'] = fip_r_tot
pitchers['xfip_r_tot'] = xfip_r_tot

In [36]:
pitchers

,team code,game_date,game_number,mlb id,Name,batting order,confirmed,position,num_L_bats,num_R_bats,...,hr_9_r_tot,k_pct_r_tot,bb_pct_r_tot,k_bb_pct_r_tot,bat_avg_r_tot,whip_r_tot,babip_r_tot,lob_pct_r_tot,fip_r_tot,xfip_r_tot
9,SF,6/9/2021,1,641793,Zack Littell,SP,N,NaN,4,5,...,17.307692,1.000000,0.416667,0.583333,1.320755,7.307692,1.250000,4.642857,39.582368,25.215017
19,TEX,6/9/2021,1,502043,Kyle Gibson,SP,N,NaN,5,4,...,3.405405,0.949367,0.379747,0.569620,0.975265,5.351351,1.233831,3.080260,14.825011,14.548116
38,OAK,6/9/2021,1,640455,Sean Manaea,SP,N,NaN,5,4,...,3.272727,0.800000,0.200000,0.600000,1.179775,5.636364,1.410909,3.135889,13.331658,15.973182
47,CHC,6/9/2021,1,453562,Jake Arrieta,SP,N,NaN,4,4,...,5.323944,0.666667,0.282828,0.383838,1.092896,5.408451,1.202797,3.043478,18.575007,17.668701
56,SD,6/9/2021,1,506433,Yu Darvish,SP,N,NaN,4,4,...,2.215385,1.406250,0.203125,1.203125,0.836820,3.876923,1.260274,3.443709,7.724429,11.872608
65,LAD,6/9/2021,1,664062,Tony Gonsolin,SP,N,NaN,4,4,...,1.565217,1.209302,0.139535,1.069767,0.829268,3.478261,1.142857,2.580645,7.548288,12.665174
74,PIT,6/9/2021,1,542881,Tyler Anderson,SP,N,NaN,2,6,...,7.280899,1.218997,0.538259,0.680739,1.495496,7.887640,1.670588,4.387352,26.192616,30.090019
83,ATL,6/9/2021,1,656353,Tucker Davidson,SP,N,NaN,2,6,...,11.571429,0.571429,0.857143,-0.285714,1.371429,9.000000,1.200000,2.678571,41.991407,40.334653
92,PHI,6/9/2021,1,621107,Zach Eflin,SP,N,NaN,4,4,...,2.535211,1.024911,0.142349,0.882562,1.037037,4.507042,1.340206,3.013699,9.889851,11.371360
102,NYM,6/9/2021,1,592836,Taijuan Walker,SP,N,NaN,5,4,...,2.261780,0.880000,0.272000,0.608000,0.740088,3.706806,0.888889,3.049645,12.786471,16.345713


**Data Description:** This worksheet provides a framework for creating dataframes for each days' players and stats. I will combine these dataframes across many days to get my final training data. These data frames have all of the stats that I could potentially be interested in, the features will be reduced for modeling as there is a significant amount of correlation between many of the features. The batters dataframe includes the batters stats against pitchers who throw with the same hand as the opposing pitcher, and includes the stats of how the opposing pitcher performs against batters with the same handedness of the batter. The pitchers dataframe contains each pitcher and their stats against both right handed and left handed batters. It also contains the number of right and left handed batters in the opposing lineup. I engineered features that are the pitchers stats vs the handedness of a batter multiplied by the number of those handed batters in the lineup. My final feature is is_nl. This is 1 if the lineup they are facing includes a pitcher and 0 if not.

**Data Dictionary:**   
team code: The team the player plays for.  
Name: The players name  
Batting order: The order the batters hit in the lineup. Is SP if player is a pitcher.  
opp_pitch_hand(batters): The hand that the opposing pitcher throws with  
opp_pitch(batters):The name of the opposing pitcher. 
opp_x(batters): This is the format for column names of stats that the opposing pitcher has against batters of the same handedness of the batter.  
(batters): There are also many columns of stats for how the batter performs against pitchers with the same handedness of the opposing pitcher.    
num_L_bats(pitchers): The number of left handed batters in the opposing lineup.  
num_R_bats(pitchers): The number of right handed batters in the opposing lineup.  
x_(l/r) (pitchers): stats describing how the pitcher performs against either left or right handed batters.    
x_(l/r)_tot (pitchers): The stats of how a pitcher performs against both hands multiplied by the number of those handed batters in the lineup.

In [37]:
scores = pd.read_csv('../datasets/6_09_target.csv', delimiter = ';')
scores.head()

,Date,GID,MLB_ID,Name,Starter,Bat order,DK posn,DK pts,DK sal,Team,Oppt,dblhdr,Tm Runs,Opp Runs
0,20210609,118v,592836,"Walker, Taijuan",1.0,0,1.0,32.15,8600.0,NYM,@ bal,NaN,14,1
1,20210609,100u,458681,"Lynn, Lance",1.0,0,1.0,29.35,10400.0,CHW,v tor,NaN,2,6
2,20210609,110g,543037,"Cole, Gerrit",1.0,0,1.0,28.50,11200.0,NYY,@ min,NaN,9,6
3,20210609,1628,425794,"Wainwright, Adam",1.0,9,1.0,25.35,9500.0,STL,v cle,NaN,8,2
4,20210609,152u,656288,"Canning, Griffin",1.0,0,1.0,24.80,8400.0,LAA,v kan,NaN,6,1


In [38]:
def clean_names(name):
    name = name.replace(',','')
    name = name.replace('.','')
    name = name.replace('-hitter','')
    name = name.replace('-pitcher','')
    names = name.split()
    first = names.pop(-1)
    names.insert(0, first)
    name = ' '.join(names)
    name = name.replace('James Happ', 'JA Happ')
    return name

In [39]:
scores['Name'] = [clean_names(name) for name in scores['Name']]

In [40]:
pitch_s = scores[scores['DK posn'] == 1]
hit_s = scores[scores['DK posn'] != 1]

In [41]:
pitch_final = pd.merge(left = pitchers, right = pitch_s, how = 'inner')

In [42]:
bat_final = pd.merge(left = batters, right = hit_s,
         how = 'inner').dropna(how = 'any', subset = ['bats', 'DK pts', 'Starter'])

In [43]:
bat_final = bat_final[bat_final['dblhdr'].isna()]

In [44]:
bat_final.to_csv('../merged_data/hitters_6_09.csv')
pitch_final.to_csv('../merged_data/pitchers_6_09.csv')

In [45]:
bat_final

,team code,game_date,game_number,mlb id,Name,batting order,confirmed,position,opp_pitch_hand,opp_pitch,...,Starter,Bat order,DK posn,DK pts,DK sal,Team,Oppt,dblhdr,Tm Runs,Opp Runs
1,SF,6/9/2021,1,527038,Wilmer Flores,4,N,NaN,R,Kyle Gibson,...,1.0,3,34.0,2.0,4300.0,SFO,@ tex,NaN,3,4(F11)
2,SF,6/9/2021,1,543063,Brandon Crawford,5,N,NaN,R,Kyle Gibson,...,1.0,4,6.0,0.0,4000.0,SFO,@ tex,NaN,3,4(F11)
3,SF,6/9/2021,1,456781,Donovan Solano,6,N,NaN,R,Kyle Gibson,...,1.0,5,4.0,3.0,3400.0,SFO,@ tex,NaN,3,4(F11)
4,SF,6/9/2021,1,621453,Steven Duggar,7,N,NaN,R,Kyle Gibson,...,1.0,6,7.0,3.0,3100.0,SFO,@ tex,NaN,3,4(F11)
5,SF,6/9/2021,1,643565,Mike Tauchman,8,N,NaN,R,Kyle Gibson,...,1.0,9,7.0,0.0,2900.0,SFO,@ tex,NaN,3,4(F11)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
191,KC,6/9/2021,1,670032,Nicky Lopez,9,N,NaN,R,Griffin Canning,...,1.0,9,6.0,3.0,2500.0,KAN,@ laa,NaN,1,6
192,LAA,6/9/2021,1,457708,Justin Upton,1,N,NaN,R,Brad Keller,...,1.0,1,7.0,7.0,4000.0,LAA,v kan,NaN,6,1
193,LAA,6/9/2021,1,543685,Anthony Rendon,3,N,NaN,R,Brad Keller,...,1.0,3,5.0,9.0,4400.0,LAA,v kan,NaN,6,1
194,LAA,6/9/2021,1,665120,Jared Walsh,4,N,NaN,R,Brad Keller,...,1.0,4,3.0,7.0,4100.0,LAA,v kan,NaN,6,1


In [46]:
pitch_final

,team code,game_date,game_number,mlb id,Name,batting order,confirmed,position,num_L_bats,num_R_bats,...,Starter,Bat order,DK posn,DK pts,DK sal,Team,Oppt,dblhdr,Tm Runs,Opp Runs
0,SF,6/9/2021,1,641793,Zack Littell,SP,N,NaN,4,5,...,1.0,0,1.0,3.65,4000.0,SFO,@ tex,NaN,3,4(F11)
1,TEX,6/9/2021,1,502043,Kyle Gibson,SP,N,NaN,5,4,...,1.0,0,1.0,15.90,6900.0,TEX,v sfo,NaN,4,3(F11)
2,OAK,6/9/2021,1,640455,Sean Manaea,SP,N,NaN,5,4,...,1.0,0,1.0,21.10,8400.0,OAK,v ari,NaN,4,0
3,CHC,6/9/2021,1,453562,Jake Arrieta,SP,N,NaN,4,4,...,1.0,9,1.0,18.25,7200.0,CHC,@ sdg,NaN,3,1
4,SD,6/9/2021,1,506433,Yu Darvish,SP,N,NaN,4,4,...,1.0,9,1.0,24.75,9800.0,SDG,v chc,NaN,1,3
5,LAD,6/9/2021,1,664062,Tony Gonsolin,SP,N,NaN,4,4,...,1.0,9,1.0,5.55,9100.0,LAD,@ pit,NaN,2,1
6,PIT,6/9/2021,1,542881,Tyler Anderson,SP,N,NaN,2,6,...,1.0,9,1.0,8.70,8000.0,PIT,v lad,NaN,1,2
7,ATL,6/9/2021,1,656353,Tucker Davidson,SP,N,NaN,2,6,...,1.0,9,1.0,18.50,6200.0,ATL,@ phi,NaN,1,2
8,PHI,6/9/2021,1,621107,Zach Eflin,SP,N,NaN,4,4,...,1.0,9,1.0,19.50,8900.0,PHI,v atl,NaN,2,1
9,NYM,6/9/2021,1,592836,Taijuan Walker,SP,N,NaN,5,4,...,1.0,0,1.0,32.15,8600.0,NYM,@ bal,NaN,14,1
